In [1]:
import requests
import time
import lxml.html
from lxml.html import fromstring, tostring, HTMLParser
import pandas as pd

#### 출처 : https://skyjwoo.tistory.com/entry/%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%9B%B9%ED%88%B0-%EB%8C%93%EA%B8%80-%ED%81%AC%EB%A1%A4%EB%A7%81-3?category=903933 
주요 코드는 이 사이트에서 참조했으며, 이를 lxml 형식으로 수정, 응용해서 사용함

## 현재 연재 중인 웹툰

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 스크롤 내리기

In [3]:
base_url = 'https://comic.naver.com/webtoon/weekday.nhn' # 연재

In [4]:
#driver 실행 
def drive(url): 
    driver = webdriver.Chrome('C:/Users/leeji/ML_/chromedriver_win32/chromedriver.exe') #driver 객체 불러옴 
    driver.implicitly_wait(3) # 3초 후에 작동하도록 
    driver.get(url) # url에 접속 
    html = driver.page_source # 현재 driver에 나타난 창의 page_source(html) 가져오기 
    soup = lxml.html.fromstring(html, parser=HTMLParser(encoding='utf-8'))
    return driver, soup

In [5]:
# 웹툰 기본 페이지에서 데이터 가져오기 
driver, soup = drive(base_url) 
driver.close()

In [6]:
# 가져온 데이터 파싱, id, 요일, 이름 
title = soup.cssselect('.title')
t_IDs = list(map(lambda x: x.get('href').split('titleId=')[1].split('&')[0], title)) 
t_weekdays = list(map(lambda x: x.get('href').split('weekday=')[1], title)) 
t_names = list(map(lambda x: x.text, title)) 

# 크롤링이 잘 되었나 확인하기 위함, 총 웹툰 수 
print('t_IDs: ', len(t_IDs)) 
print('t_weekdays: ', len(t_weekdays)) 
print('t_names: ', len(t_names)) 
print(t_IDs[0], t_weekdays[0], t_names[0])

t_IDs:  473
t_weekdays:  473
t_names:  473
758037 mon 참교육


In [7]:
#웹툰 이름으로 id와 weekday 반환
def find_id_weekday(name,t_names,t_IDs,t_weekdays,start_idx = 0):
    try:
        idx = t_names.index(name)
    except:
        print('찾는 웹툰이 없습니다.')
        return
    return t_IDs[idx], t_weekdays[idx]

In [10]:
#episode 개수 세기 
def episode_count(ID): 
    url = base_url.split('finish')[0]+'list.nhn?titleId={0}'.format(ID) 
    driver, soup = drive(url) 
    driver.close()
    res = soup.cssselect('.title a')[0].get('href').split('no=')[1].split('&')[0] 
    return res 

res = episode_count(183559) # 신의 탑 
print(res)

IndexError: list index out of range

In [8]:
import pandas as pd

def comment_crawler(name, start_idx=0):
    id_num, weekday = find_id_weekday(name,t_names,t_IDs,t_weekdays, start_idx=start_idx)
    cnt = int(episode_count(id_num, weekday))
    
    comments = pd.DataFrame(index=range(0, cnt+1), columns=['comment'])
    comment_ele = []
    score = []
    proceed = -1 #진행 상태 표시 위함, 처음에 0보다 작아야 0%가 표시 됨
    
    driver, _ = drive(base_url) #driver만 먼저 열어 놓음. for문 돌면서 url만 바꿔줄 것임
    print('진행중...')
    for i in range(1, cnt+1):
        comment_ele = []
        percentage = int((i/cnt)*100)
        if percentage%10==0 and percentage>proceed: # 진행상황 표시
            proceed = percentage
            print(proceed, '% 완료')
        url = 'https://comic.naver.com/comment/comment.nhn?titleId={0}&no={1}#'.format(id_num, str(i))
        time.sleep(1.5)
        driver.get(url)
        html = driver.page_source
        soup = lxml.html.fromstring(html, parser=HTMLParser(encoding='utf-8'))
        
        comment_ele += list(map(lambda x: x.text, soup.cssselect('.u_cbox_contents')))
        comments['comment'][i] = comment_ele
             
    driver.close()
    print('crawling finished')
    return comments

In [9]:
comments = comment_crawler('신의 탑')

NameError: name 'episode_count' is not defined

In [28]:
# 수집된 댓글 수 
print(len(comments))

123


In [29]:
comments

,comment
0,NaN
1,"[아무생각없이 내렸다가 이거 보고 놀란사람 손!!!, 헐?, 제가 이 웹툰을 보면서..."
2,"[맨 처음에 온 교수 프로스트 줄 알았다 손, 헐 닥터 프로스트가 저렇게 된거임....."
3,"[어쩌라고 심리학자본능이 있는데, 프로스트가! 그냥! 환자래! 이런 미친!, 천교수..."
4,[설명충이라서 죄송하지만 송설은 우울증환자로 프로스트 첫번째 환자라고 보면 됩니다 ...
...,...
118,[겁나 불안하네.......초반부 저 장면이 프롤로그의 환장 파티로 이어지는건 아니...
119,"[""썸""""결정적인 한방이 부족해""이건 누가봐도 한국 토박이인데?, 매니가 남봉성아 ..."
120,[문성현.. ' 그녀에 대해 들었을 때 어땠어? ' 라는 말하면서 짓는 저 표정. ...
121,"[정말 현 시대, 대한민국을 찌르는 명화네요.. 예전부터 느꼈지만 동시대를 살아가는..."


In [30]:
comments.to_csv('닥터프로스트_comments.csv')

In [14]:
def crawler(name, start_idx=0):
    id_num, weekday = find_id_weekday(name,t_names,t_IDs,t_weekdays, start_idx=start_idx)
    cnt = int(episode_count(id_num, weekday))
    
    scores = []
    episodes = []
    dates = []
    proceed = -1 #진행 상태 표시 위함, 처음에 0보다 작아야 0%가 표시 됨
    
    driver, _ = drive(base_url) #driver만 먼저 열어 놓음. for문 돌면서 url만 바꿔줄 것임
    print('진행중...')
    for i in range(1, cnt+1):
        percentage = int((i/cnt)*100)
        if percentage%10==0 and percentage>proceed: # 진행상황 표시
            proceed = percentage
            print(proceed, '% 완료')

        url = 'https://comic.naver.com/webtoon/detail.nhn?titleId={0}&no={1}&weekday={2}'.format(id_num, str(i), weekday)
        time.sleep(1.5)
        driver.get(url)
        html = driver.page_source 
        soup = lxml.html.fromstring(html, parser=HTMLParser(encoding='utf-8'))
        
        scores += list(map(lambda x : x.text, soup.cssselect('div.rating_type4 span#topPointTotalNumber')[0]))
        episodes += list(map(lambda x : x.text, soup.cssselect('div.view h3')))
        dates += list(map(lambda x : x.text, soup.cssselect('dl.rt dd.date')))
                
    driver.close()
    print('crawling finished')
    return scores, episodes, dates

In [31]:
scores, episodes, dates = crawler('닥터 프로스트 시즌 3~4')

진행중...
0 % 완료
10 % 완료
20 % 완료
30 % 완료
40 % 완료
50 % 완료
60 % 완료
70 % 완료
80 % 완료
90 % 완료
100 % 완료
crawling finished


In [32]:
print(len(scores), len(episodes), len(dates))

122 122 122


In [33]:
sed = pd.DataFrame({'score':scores, 'episode':episodes, 'date':dates})
sed.to_csv('닥터프로스트_sed.csv'); sed

,score,episode,date
0,9.95,예고편,2015.06.02
1,9.97,301-File no.#15 닫힌 방의 열린 문(1),2015.06.09
2,9.98,302-File no.#15 닫힌 방의 열린 문(2),2015.06.16
3,9.98,303-File no.#15 닫힌 방의 열린 문(3),2015.06.23
4,9.98,304-File no.#15 닫힌 방의 열린 문(4),2015.06.30
...,...,...,...
117,9.97,475-Ep9. 끝의 시작(1),2021.03.18
118,9.97,476-Ep9. 끝의 시작(2),2021.03.25
119,9.97,477-Ep9. 끝의 시작(3),2021.04.01
120,9.95,478-Ep9. 끝의 시작(4),2021.04.08


## 완결 or 휴재인 웹툰

In [3]:
base_url = 'https://comic.naver.com/webtoon/finish.nhn?order=User&view=image' # 완결 or 휴재

In [4]:
#driver 실행 
def drive(url): 
    driver = webdriver.Chrome('C:/Users/leeji/ML_//chromedriver_win32/chromedriver.exe') #driver 객체 불러옴 
    driver.implicitly_wait(3) # 3초 후에 작동하도록 
    driver.get(url) # url에 접속 
    html = driver.page_source # 현재 driver에 나타난 창의 page_source(html) 가져오기 
    soup = lxml.html.fromstring(html, parser=HTMLParser(encoding='utf-8'))
    return driver, soup

In [5]:
# 웹툰 기본 페이지에서 데이터 가져오기 
driver, soup = drive(base_url) 
driver.close()

In [6]:
#웹툰 이름으로 id 반환
def find_id(name,t_names,t_IDs,start_idx = 0):
    try:
        idx = t_names.index(name)
    except:
        print('찾는 웹툰이 없습니다.')
        return
    return t_IDs[idx]

In [7]:
# 가져온 데이터 파싱, id, 요일, 이름 
title = soup.cssselect('li dt a')
t_IDs = list(map(lambda x: x.get('href').split('titleId=')[1].split('&')[0], title)) 
#t_weekdays = list(map(lambda x: x.get('href').split('weekday=')[1], title)) 
t_names = list(map(lambda x: x.text.strip(), title)) 

# 크롤링이 잘 되었나 확인하기 위함, 총 웹툰 수 
print('t_IDs: ', len(t_IDs)) 
#print('t_weekdays: ', len(t_weekdays)) 
print('t_names: ', len(t_names)) 
print(t_IDs[0], t_names[0])

t_IDs:  921
t_names:  921
690503 도망자


In [8]:
#episode 개수 세기 
def episode_count(ID): 
    url = base_url.split('finish')[0]+'list.nhn?titleId={0}'.format(ID) 
    driver, soup = drive(url) 
    driver.close()
    res = soup.cssselect('.title a')[0].get('href').split('no=')[1].split('&')[0] 
    return res 

res = episode_count(183559) # 신의 탑 
print(res)

487


In [9]:
def comment_crawler(name, start_idx=0):
    id_num = find_id(name, t_names, t_IDs, start_idx=start_idx)
    cnt = int(episode_count(id_num))
    
    comments = pd.DataFrame(index=range(0, cnt+1), columns=['comment'])
    comment_ele = []
    score = []
    proceed = -1 #진행 상태 표시 위함, 처음에 0보다 작아야 0%가 표시 됨
    
    driver, _ = drive(base_url) #driver만 먼저 열어 놓음. for문 돌면서 url만 바꿔줄 것임
    print('진행중...')
    for i in range(1, cnt+1):
        comment_ele = []
        percentage = int((i/cnt)*100)
        if percentage%10==0 and percentage>proceed: # 진행상황 표시
            proceed = percentage
            print(proceed, '% 완료')
        url = 'https://comic.naver.com/comment/comment.nhn?titleId={0}&no={1}#'.format(id_num, str(i))
        time.sleep(1.5)
        driver.get(url)
        html = driver.page_source
        soup = lxml.html.fromstring(html, parser=HTMLParser(encoding='utf-8'))
        
        comment_ele += list(map(lambda x: x.text, soup.cssselect('.u_cbox_contents')))
        comments['comment'][i] = comment_ele
             
    driver.close()
    print('crawling finished')
    return comments

In [10]:
comments = comment_crawler('덴마') 
# 신의탑, 대학일기, 놓지마 정신줄, 스피릿 핑거스, 덴마

진행중...
0 % 완료
10 % 완료
20 % 완료
30 % 완료
40 % 완료
50 % 완료
60 % 완료
70 % 완료
80 % 완료
90 % 완료
100 % 완료
crawling finished


In [11]:
# 수집된 댓글 수 
print(len(comments))

1418


In [12]:
comments

,comment
0,NaN
1,[2차 정주행. 그리고 3년정도 우렸으면 이제 저 사람 안꼬이는 메인 섬네일 좀 바...
2,"[이때 난 몰랐지 이게 얼마나 재미있는건지, 초반엔 개념 잡는데도 한참걸려서 대체..."
3,"[댓글이 적어지는이유가 몰입이너무 세서.., 아내가 폰을 숨기는거 봐선 이미 바람난..."
4,[마지막 저 아버지의 표정변화씬은... 정말 할말이 없을정도로 쩌는듯... 저런 미...
...,...
1413,"[불평불만 하면서 매번 바로 들어오는 덴경대가 레전드, 뭔 우주 최강의 퀑이 저런 ..."
1414,"[자꾸 보러오는 우리가 제일 문제 아닐까, 낭심ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 저번에 누가 헤글..."
1415,"[그러든가 말든가 ㅋㅋㅋㅋㅋㅋㅋ작가의 말 저기다 써놨네, 다음편 에드레이의 박수 그..."
1416,"[덴마가 컸으니 덴마크 앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ, 아니 씨x 귀여운 셀 어디가고 저런 ..."


In [13]:
comments.to_csv('덴마_comments.csv')

In [14]:
def crawler(name, start_idx=0):
    id_num = find_id(name, t_names, t_IDs, start_idx=start_idx)
    cnt = int(episode_count(id_num))
    
    scores = []
    episodes = []
    dates = []
    proceed = -1 #진행 상태 표시 위함, 처음에 0보다 작아야 0%가 표시 됨
    
    driver, _ = drive(base_url) #driver만 먼저 열어 놓음. for문 돌면서 url만 바꿔줄 것임
    print('진행중...')
    for i in range(1, cnt+1):
        percentage = int((i/cnt)*100)
        if percentage%10==0 and percentage>proceed: # 진행상황 표시
            proceed = percentage
            print(proceed, '% 완료')

        url = 'https://comic.naver.com/webtoon/detail.nhn?titleId={0}&no={1}'.format(id_num, str(i))
        time.sleep(1.5)
        driver.get(url)
        html = driver.page_source 
        soup = lxml.html.fromstring(html, parser=HTMLParser(encoding='utf-8'))
        
        scores += list(map(lambda x : x.text, soup.cssselect('div.rating_type4 span#topPointTotalNumber')[0]))
        episodes += list(map(lambda x : x.text, soup.cssselect('div.view h3')))
        dates += list(map(lambda x : x.text, soup.cssselect('dl.rt dd.date')))
                
    driver.close()
    print('crawling finished')
    return scores, episodes, dates

In [15]:
scores, episodes, dates = crawler('덴마')

진행중...
0 % 완료
10 % 완료
20 % 완료
30 % 완료
40 % 완료
50 % 완료
60 % 완료
70 % 완료
80 % 완료
90 % 완료
100 % 완료
crawling finished


In [16]:
print(len(scores), len(episodes), len(dates))

1417 1417 1417


In [17]:
sed = pd.DataFrame({'score':scores, 'episode':episodes, 'date':dates})
sed.to_csv('덴마_sed.csv'); sed

,score,episode,date
0,8.93,1화 파마나의 개(1),2010.01.08
1,9.27,2화 파마나의 개(2),2010.01.10
2,9.38,3화 파마나의 개(3),2010.01.11
3,9.53,4화 파마나의 개(4),2010.01.15
4,9.46,5화 해적선장 하독(1),2010.01.17
...,...,...,...
1412,5.06,3-410화 2.에필로그(4-7),2019.12.22
1413,6.24,3-411화 2.에필로그(4-8),2019.12.24
1414,6.32,3-412화 2.에필로그(4-9),2019.12.25
1415,5.12,3-413화 2.에필로그(5-1),2019.12.27
